In [8]:
import os

import dotenv

from time import time, sleep
from algosdk.future import transaction
from algosdk.logic import get_application_address

from bidding.operations import *
from store.operations import StoringPool
from utils import *
from account import Account

In [9]:
dotenv.load_dotenv('.env')

client = get_algod_client(os.environ.get('ALGOD_URL'), os.environ.get('ALGOD_TOKEN'))

In [10]:
creator = Account.from_mnemonic(os.environ.get('CREATOR_MN'))
bidder = Account.from_mnemonic(os.environ.get('BUYER_MN'))
seller = Account.from_mnemonic(os.environ.get('SELLER_MN'))

storeAppID = int(os.environ.get("STORE_APP_ID"))
# close_bidding(client, 693, storeAppID, creator)
print(f"storeAppID: {storeAppID}")

print(f"Creator address: {creator.get_address()}")
print(f"Bidder address: {bidder.get_address()}")
print(f"Seller Address: {seller.get_address()}")

storeAppID: 3310
Creator address: 4TT75274EBUAF46CITUL6HQQ4C4D3GO7GEOVRZSQZ35VXSGRVHJ376GD64
Bidder address: X7PMCPPRD2T35ZZRTPETYK53G5YWBKUNONVDY52WODP3N3YM7NGEF5HGWU
Seller Address: CN4EQ6QXFCDLR2ATDTRJMMFGLXSUS5TMLGLO7FJDBQRQGFUXTUA4LBVEIA


In [11]:
deleteApps(client, [3969, 3970, 3972, 3978, 3984, 4008, 4056, 4059, 4062, 4064], creator)

Waiting for confirmation...
Transaction ZDEWBVZU77CIC6KB4QQOGLS2SUGEDG7KP6GMRZMYSJQOLKLIJZXA confirmed in round 156974.
Waiting for confirmation...
Transaction HAQUCJFCJJYWMSWH35BQO4FIKKKFGMPK3SCHYQVNJECKI3RN5RJA confirmed in round 156976.
Waiting for confirmation...
Transaction ELCT2PKHD5YBEZT3O57HYRBOB7YODJYPC5EVFFQXYP47DWMM42OA confirmed in round 156978.
Waiting for confirmation...
Transaction D64C5X62USA37BWSYFKY23SEE5DW3Q4HJBIVMFAV3M4R5XSQN4QA confirmed in round 156980.
Waiting for confirmation...
Transaction EPNQBKC5FOZ3WGCJ7VQJRFQAAZIPSEI5QWZJIVUOIK47R2EUR7NQ confirmed in round 156982.
Waiting for confirmation...
Transaction ZNWNRHY4UUX745IG2P44G3GYFPY3SUVUJX4ISYYT5LBUBFIW4NFQ confirmed in round 156984.
Waiting for confirmation...
Transaction GE2EC5DRYBFIGQYZQNZWUMOKMD3MYIJ3JX4RD7RN4V3PPFAADFSA confirmed in round 156986.
Waiting for confirmation...
Transaction DDLHYRBECJWQMNWP7FFY2OXLSQ4PMUIKBVIHLCQF425DPCRPBDJA confirmed in round 156988.
Waiting for confirmation...
Transaction 

In [12]:
#n_pri_key, n_address = generate_rekeyed_algorand_keypair(client, bidder)
# n_address = 'FOWXDCFQJKY3ISR7A5ZC2K5QEA5EYOJYCML55JWZVZYWWKQSUD6UFWIWKU'
# set_rekeyed_address(bidder.get_address(), n_address, 1)

In [13]:
# def create_dummy_asset(sender: Account, total: int, decimals: int, asset_name: str, unit_name: str):
#     txn = transaction.AssetConfigTxn(
#         sender=sender.get_address(),
#         sp=client.suggested_params(),
#         total=total,
#         decimals=decimals,
#         asset_name=asset_name,
#         unit_name=unit_name,
#         default_frozen=False,
#         strict_empty_address_check=False,
#     )
#     signed_txn = txn.sign(sender.get_private_key())

#     client.send_transaction(signed_txn)

#     response = wait_for_confirmation(client, signed_txn.get_txid())
#     assert response.asset_index is not None and response.asset_index > 0
#     return response.asset_index

# asset_id = create_dummy_asset(seller, 1, 0, "example token", "AVT")

asset_id = int(os.environ.get("TOKEN_ID"))
print(f"The token id is: {asset_id}")

The token id is: 2989


In [14]:
start_time = int(time()) + 10
end_time = start_time + 10
price = 1_000_000  # 1 Algo

"Alice is creating auction smart contract that lasts 30 seconds to auction off token..."

In [15]:
storeApp = StoringPool(client=client, creator=creator)
# storeApp.create_app()
# storeAppID = storeApp.app_id

storeAppID = int(os.environ.get("STORE_APP_ID"))
storeApp.app_id = storeAppID
# print(f"Store App ID: {storeAppID}")

storeAppAddress = get_application_address(storeAppID)
print(f"Store App Address: {storeAppAddress}")


Store App Address: ZSKDDVBDYT3U6TVCHVNGHUS74IAFMQANNO353SE6GHQMAXC3LTX5RQPEBY


In [16]:

appID = create_bidding_app(
    client=client,
    creator=creator,
    store_app_id=storeAppID
)
print(f"App ID: {appID}")
print(f"App Address: {get_application_address(appID)}")

Waiting for confirmation...
Transaction ML73A6T35V5MLCNHDTL2KAZ3M2QDSMQVPFZWON6V2V77B57SJXIQ confirmed in round 156994.
Waiting for confirmation...
Transaction ONUMVUHZOOST6MSABEOFFK3IFFCMQCXZLNLNIXPJZSYHYT3PFZ2Q confirmed in round 156996.
App ID: 4076
App Address: KDSB5G5KTXIETW32WBS5AXML2FZINSSP6CWXWFEO47PLA66RILLLBGINFA


Alice is setting up and funding token auction...

In [18]:
sellerAlgosBefore = get_balances(client, seller.get_address())[0]

print("Alice's algo balance: ", sellerAlgosBefore, " algos")

Alice's algo balance:  20077168838585  algos


In [19]:
sleep(5)
actualAppBalancesBefore = get_balances(client, get_application_address(appID))
print("The smart contract now holds the following:", actualAppBalancesBefore)

The smart contract now holds the following: {0: 100000}


In [20]:
bidPrice = price
bidderAlgosBefore = get_balances(client, bidder.get_address())[0]
print("Carla wants to bid on token, her algo balance: ", bidderAlgosBefore, " algos")
print("Carla is placing bid for: ", bidPrice, " algos")

Carla wants to bid on token, her algo balance:  10036489901092  algos
Carla is placing bid for:  1000000  algos


In [21]:
bid_index = place_bid(client=client, app_id=appID, token_id=asset_id, bidder=bidder, bid_amount=10, bid_price=bidPrice, bid_index="")

print("Carla bid", asset_id)

store_app_id 3310
new address: XYQFYWKL3WZZRDCAUPPGNNV7B55V3GXSYXBRANSR5S3GRWSXGBKZJYDV2Q
new private_key: mfA6NlWbJUigbDrLzVwM1zqULr9IUFpuHr88cZcLabO+IFxZS92zmIxAo95mtr8Pe12a8sXDEDZR7LZo2lcwVQ==
new passphrase: basket type only relief enroll bamboo govern denial ride industry mail pulp chronic slight echo expect slender keen garbage because confirm drill opera absent silent
Waiting for confirmation...
Transaction PMSBMHSMQTSZYHPUOGDJ3MIHM6QZAO2Q6YN3RLQSCIHOILSGS4GQ confirmed in round 157001.
Waiting for confirmation...
Transaction D7CNGBW3ORFWGCVXF3YWVJMF5Z4EDGSZHZ3NXA5DY3BCDQYFA42A confirmed in round 157003.
Waiting for confirmation...
Transaction GCGF4AWU7AYFBKIMHFLIB3357UBXVCI2M4MLM7HEERXD6RFFBSUA confirmed in round 157005.
Waiting for confirmation...
Transaction SGJZJHCV5WV7SAIPHCNB4WLNCRUIME5DTTJGWA6OEITWYJ3N6T2A confirmed in round 157007.
Carla bid 2989


In [ ]:
actualAppBalancesBefore = get_balances(client, get_application_address(appID))
print("The smart contract now holds the following:", actualAppBalancesBefore)

In [ ]:
# account_info = client.account_info(bidder.get_address())  
# print(account_info.get('apps-local-state', []))
# print(is_opted_in_app(client, appID, bidder.get_address()))

# cancel_bid(client, appID, bidder, bid_index)

# print("Carla canceled bid", asset_id)

In [ ]:
# bid_index = place_bid(client=client, app_id=appID, bidder=bidder, token_id=asset_id, bid_amount=10, bid_price=bidPrice, bid_index="")
# print(bid_index)
# bid_index = place_bid(client=client, app_id=appID, bidder=bidder, token_id=asset_id, bid_amount=20, bid_price=bidPrice+1_000_000, bid_index=bid_index)
# print(bid_index)
# bid_index = place_bid(client=client, app_id=appID, bidder=bidder, token_id=asset_id, bid_amount=15, bid_price=bidPrice+500_000, bid_index=bid_index)
# print(bid_index)


In [ ]:
bidderAppLocalState = get_app_local_state(client, appID, bid_index)
print(bidderAppLocalState)

In [ ]:
place_accept(client, creator, appID, seller, bidder.get_address(), bid_index)

Alice is closing out the bidding contract....

In [ ]:
sleep(5)

close_bidding(client, appID, creator, [asset_id])

In [ ]:
actualAppBalances = get_balances(client, get_application_address(appID))
expectedAppBalances = {0: 0}
print("The smart contract now holds the following:", actualAppBalances)

#assert actualAppBalances == expectedAppBalances

In [ ]:
bidderNftBalance = get_balances(client, bidder.get_address())[asset_id]

print("Carla's NFT balance:", bidderNftBalance, " for token ID: ", asset_id)

assert bidderNftBalance == 1

actualSellerBalances = get_balances(client, seller.get_address())
print("Alice's balances after auction: ", actualSellerBalances, " Algos")

actualBidderBalances = get_balances(client, bidder.get_address())
print("Carla's balances after auction: ", actualBidderBalances, " Algos")
# assert len(actualSellerBalances) == 2
# seller should receive the bid amount, minus the txn fee
assert actualSellerBalances[0] >= sellerAlgosBefore + bidPrice - 1_000
assert actualSellerBalances[asset_id] == 0

In [ ]:
storeAppState = get_app_global_state(client, 1676)
print(storeAppState)
#print(encoding.encode_address(storeAppState[b"C"]))